# Step 2 - Prepare network data
## Project: Growing Urban Bicycle Networks

This notebook downloads cycle, LTN, and street network data sets and prepares them for analysis.

Contact: Chris Larkin (c.larkin@ncl.ac.uk) or Michael Szell (michael.szell@gmail.com)
 
Created: 2024-11-16  
Last modified: 2024-11-17

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%load_ext watermark
#%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Download and wrangle data

### Networks

In [ ]:
for scenario in ["no_ltn_scenario",
"current_ltn_scenario","more_ltn_scenario"]:
    for placeid, placeinfo in tqdm(cities.items(), desc=f"Cities ({scenario})"):
        base_path = os.path.join(PATH["data"], placeid, scenario)

        if placeinfo["nominatimstring"] != '':
            location = ox.geocoder.geocode_to_gdf(placeinfo["nominatimstring"])
            if location.geometry[0].geom_type == 'MultiPolygon':
                location = location.explode(index_parts=False).reset_index(drop=True)
            location = fill_holes(extract_relevant_polygon(placeid, shapely.geometry.shape(location['geometry'][0])))
            if debug:
                try:
                    color = cm.rainbow(np.linspace(0, 1, len(location)))
                    for poly, c in zip(location, color):
                        plt.plot(*poly.exterior.xy, c=c)
                        for intr in poly.interiors:
                            plt.plot(*intr.xy, c="red")
                except:
                    plt.plot(*location.exterior.xy)
                plt.show()
        else:
            shp = fiona.open(PATH["data"] + placeid + "/" + placeid + ".shp")
            first = next(iter(shp))
            try:
                location = Polygon(shapely.geometry.shape(first['geometry']))
            except:
                location = shapely.geometry.shape(first['geometry'])

        Gs = {}
        for parameterid, parameterinfo in tqdm(osmnxparameters.items(), desc="Networks", leave=False):
            for i in range(10):
                try:
                    Gs[parameterid] = ox.graph_from_polygon(
                        location,
                        network_type=parameterinfo['network_type'],
                        custom_filter=parameterinfo['custom_filter'],
                        retain_all=parameterinfo['retain_all'],
                        simplify=False
                    )
                    break
                except ValueError:
                    Gs[parameterid] = nx.empty_graph(create_using=nx.MultiDiGraph)
                    print(placeid + ": No OSM data for graph " + parameterid + ". Created empty graph.")
                    break
                except (ConnectionError, UnboundLocalError):
                    print("ConnectionError or UnboundLocalError. Retrying.")
                    continue
                except:
                    print("Other error. Retrying.")
                    continue
            if parameterinfo['export']:
                ox_to_csv(Gs[parameterid], base_path + "/", placeid, parameterid)
                ox.save_graph_geopackage(Gs[parameterid], filepath=base_path + "/" + placeid + "_" + parameterid + ".gpkg", directed=False)

        # Load and process LTN neighbourhoods (skip for no_ltn_scenario)
        city_neighbourhood_streets = {}
        if scenario != "no_ltn_scenario":
            neighbourhoods = load_neighbourhoods(base_path + "/")
            if not neighbourhoods:
                print(placeid + ": No LTN dataset found.")
            else:
                neighbourhoods = prepare_neighbourhoods(neighbourhoods)
                for city_name, gdf in neighbourhoods.items():
                    if debug:
                        print(f"Processing streets for {city_name}...")
                    # Patch cycle graphs with pedestrian links and get results
                    results = patch_cycle_graph_with_pedestrian_links(neighbourhoods, debug=debug)
                    for city_name, stats in results.items():
                        patched_edges = stats.get("patched_edges")
                        if not patched_edges.empty:
                            patched_edges.to_file(os.path.join(PATH["data"], placeid, scenario, f"{placeid}_patched_edges.gpkg"), driver='GPKG'),
                        nodes, edges = ox.graph_to_gdfs(stats['cycle_graph_after'])
                        city_neighbourhood_streets[city_name] = {
                            'nodes': nodes,
                            'edges': edges,
                            'neighbourhood_graphs': stats['cycle_graph_after']}
                    # Plot and save patching stats & CSV
                    output_plot_path = os.path.join(PATH["plots"] + "/" + placeid + "/" + scenario + "/" + f"{placeid}_pedestrian_paths_in_ltn_analysis.png")
                    output_csv_path = os.path.join(PATH["results"], placeid, scenario, "pedestrian_in_ltn.csv")
                    plot_and_save_network_stats(results, output_plot_path, output_csv_path, scenario)

        # Compose graphs
        if scenario != "no_ltn_scenario" and city_name in city_neighbourhood_streets:
            neighbourhood_graph = city_neighbourhood_streets[city_name]['neighbourhood_graphs']
            neighbourhood_graph = clean_edge_attributes(neighbourhood_graph)
            Gs['biketrack'] = nx.compose_all([
                Gs['bike_cyclewaylefttrack'], Gs['bike_cyclewaytrack'],
                Gs['bike_highwaycycleway'], Gs['bike_bicycleroad'],
                Gs['bike_cyclewayrighttrack'], Gs['bike_designatedpath'],
                Gs['bike_cyclestreet'], neighbourhood_graph
            ])
        else:
            Gs['biketrack'] = nx.compose_all([
                Gs['bike_cyclewaylefttrack'], Gs['bike_cyclewaytrack'],
                Gs['bike_highwaycycleway'], Gs['bike_bicycleroad'],
                Gs['bike_cyclewayrighttrack'], Gs['bike_designatedpath'],
                Gs['bike_cyclestreet']
            ])

        parameterid = 'biketrack'
        ox_to_csv(Gs[parameterid], base_path + "/", placeid, parameterid)
        ox.save_graph_geopackage(Gs[parameterid], filepath=base_path + "/" + placeid + "_" + parameterid + ".gpkg", directed=False)

        parameterid = 'bikeable'
        Gs[parameterid] = nx.compose_all([Gs['biketrack'], Gs['car30'], Gs['bike_livingstreet']])
        ox_to_csv(Gs[parameterid], base_path + "/", placeid, parameterid)
        ox.save_graph_geopackage(Gs[parameterid], filepath=base_path + "/" + placeid + "_" + parameterid + ".gpkg", directed=False)

        parameterid = 'biketrackcarall'
        Gs[parameterid] = nx.compose(Gs['biketrack'], Gs['carall'])
        ox_to_csv(Gs[parameterid], base_path + "/", placeid, parameterid)
        ox.save_graph_geopackage(Gs[parameterid], filepath=base_path + "/" + placeid + "_" + parameterid + ".gpkg", directed=False)

        parameterid = 'biketrack_no_ltn'
        Gs[parameterid] = nx.compose_all([
            Gs['bike_cyclewaylefttrack'], Gs['bike_cyclewaytrack'],
            Gs['bike_highwaycycleway'], Gs['bike_bicycleroad'],
            Gs['bike_cyclewayrighttrack'], Gs['bike_designatedpath'],
            Gs['bike_cyclestreet']
        ])
        ox_to_csv(Gs[parameterid], base_path + "/", placeid, parameterid)
        ox.save_graph_geopackage(Gs[parameterid], filepath=base_path + "/" + placeid + "_" + parameterid + ".gpkg", directed=False)

        for parameterid in networktypes[:-2]:
            ox_to_csv(ox.simplify_graph(Gs[parameterid]), base_path + "/", placeid, parameterid, "_simplified")


In [ ]:
# Compress all data files (will not do anything if files were compressed already)
for folder, subfolders, files in os.walk(PATH["data"]):
    for file in files:
        if file.endswith('es.csv'):
            compress_file(folder + "/", file.split(".")[0])

In [ ]:
Audio(sound_file, autoplay=True)